# Geocoding Toronto neighbors

## Using a geocoding tool

**geocode** library for python does not work correctly due to the actual google maps restriction I guess. Nevertheless there are other tools for geocoding that work perfectly with openmaps like **geopy** library.

Here we make a function **nominatim_geocode** which returns a the geocoding object from any given adress. The function is asking the server until getting the cordinates: 

In [20]:
from geopy.geocoders import Nominatim
nominatim = Nominatim(user_agent="my-application2")

from time import sleep
from geopy.exc import GeocoderTimedOut
def nominatim_geocode(address):
    try:
        return nominatim.geocode(query=address)
    except GeocoderTimedOut:
        print('.')
        sleep(15)
        return nominatim_geocode(address)

Testing the function:

In [21]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
quer = nominatim_geocode('Mountain View, CA')
lat_lng_coords = [quer.latitude, quer.longitude]

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print(latitude,', ',longitude)

.
.
.
.
.
.
.
.
51.6878246 ,  -114.2502063


## Import Toronto Postal codes

In [24]:
import pandas as pd
postal_codes = pd.read_csv('postal_codes.csv',index_col =0)

In [25]:
postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [26]:
postal_codes.shape

(103, 3)

Create new datafreme and add the column Latitude and Longitude

In [27]:
geo_postal_codes = pd.DataFrame(columns = list(postal_codes.columns) + ['Latitude', 'Longitude'])
geo_postal_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude


## Geocoding

In [28]:
#import a library for controling the Ipython output
from IPython.display import clear_output, display

In [32]:
for index, row in postal_codes.iterrows():
    clear_output(wait=True)
    display(str(index) + ' of ' + str(len(postal_codes)))
    address = str(row['Borough']) + ', Toronto, Canada'
    print(address)
    g = nominatim_geocode(address)
    print(g.latitude,', ',g.longitude)
    
    #fill the dataframe
    geo_postal_codes = geo_postal_codes.append({'PostalCode': row.PostalCode,
                                                'Borough': row.Borough,
                                                'Neighborhood': row.Neighborhood,
                                                'Latitude': g.latitude,
                                                'Longitude': g.longitude}, ignore_index=True)
    
    #sleep for not saturate the geocoding server
    sleep(1)

'102 of 103'

Etobicoke, Toronto, Canada
43.671459150000004 ,  -79.55249206611668


In [33]:
geo_postal_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.754326,-79.449117
1,M4A,North York,Victoria Village,43.754326,-79.449117
2,M5A,Downtown Toronto,Harbourfront,43.656322,-79.380916
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.754326,-79.449117
4,M7A,Downtown Toronto,Queen's Park,43.656322,-79.380916


In [34]:
geo_postal_codes.shape

(103, 5)

In [ ]:
#Saving the data
geo_postal_codes.to_csv('postal_codes_latlon.csv')